In [1]:
import zmq
import random
import sys
from random import randint
from threading import Thread
from time import sleep

Option A

- instead of sending messages directly, we first store them in the store, and then have some kind of daemon that listens to changes in the store, and puts them in the queue on the main worker


Option B
- instead of keeping the connection to zmq open all the time as producer, we open the connection, send the data, and close. And save the producer address as part of the queueconfig

In [2]:
import zmq

In [3]:
# zmq.proxy(frontend, backend)

In [4]:

# # we never get here
# frontend.close()
# backend.close()
# context.term()

In [5]:
def run_message_queue():
#     context = zmq.Context()

#     # Socket facing clients
#     frontend = context.socket(zmq.ROUTER)
#     frontend.bind("tcp://*:5559")

#     # Socket facing services
#     backend  = context.socket(zmq.DEALER)
#     backend.bind("tcp://*:5560")
#     print("starting queue")
    
    
#     zmq.proxy(frontend, backend)
#     # we never get here
#     frontend.close()
#     backend.close()
#     context.term()

    # Prepare our context and sockets
    context = zmq.Context()
    frontend = context.socket(zmq.ROUTER)
    backend = context.socket(zmq.DEALER)
    frontend.bind("tcp://*:5559")
    backend.bind("tcp://*:5560")

    # Initialize poll set
    poller = zmq.Poller()
    poller.register(frontend, zmq.POLLIN)
    poller.register(backend, zmq.POLLIN)

        # Switch messages between sockets
    while True:
        socks = dict(poller.poll())

        if socks.get(frontend) == zmq.POLLIN:
            message = frontend.recv_multipart()
            backend.send_multipart(message)

        if socks.get(backend) == zmq.POLLIN:
            message = backend.recv_multipart()
            frontend.send_multipart(message)


In [6]:
def run_producer():
    port = "5559"

    context = zmq.Context()
    socket = context.socket(zmq.REQ)
    socket.connect(f"tcp://localhost:{port}")

    sleep(1)
    for i in range(3):
        topic = random.randrange(9999,10005)
        messagedata = random.randrange(1,215) - 80
        print("sending message")
        socket.send(b'1111')
#         message = socket.recv()
        sent=True
        sleep(1)            
    socket.close()

In [7]:
def run_consumer():
    port = "5560"

    print("starting consumer")
    c_id = randint(0,100)

    # Socket to talk to server
    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.connect ("tcp://localhost:%s" % port)
    n_messages=0
    while True:
        print("listening")
        string = socket.recv()
#         socket.send(b'')
        n_messages+=1
        print(f"received {string} on consumer {c_id} n={n_messages}")
        sleep(3)


In [8]:
Thread(target=run_consumer).start()
Thread(target=run_consumer).start()
Thread(target=run_producer).start()
Thread(target=run_producer).start()
Thread(target=run_message_queue).start()

sleep(60)

starting consumerstarting consumer
listening

listening
sending message
sending message
received b'1111' on consumer 50 n=1
received b'1111' on consumer 58 n=1


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/ipykernel_27526/4150166507.py", line 13, in run_producer
    self.run()
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/ipykernel_27526/4150166507.py", line 13, in run_producer
  File "/opt/anaconda3/envs/syft/lib/python3.9/site-packages/zmq/sugar/socket.py", line 696, in send
  File "/opt/anaconda3/envs/syft/lib/python3.9/site-packages/zmq/sugar/socket

sending messagesending message



Exception in thread Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Thread-6:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 917, in run
    self.run()
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/ipykernel_27526/2140229155.py", line 14, in run_consumer
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/ipykernel_27526/2140229155.py", line 14, in run_consumer
  File "zmq/backend/cython/socket.pyx", line 805, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 805, in zmq.backend.cython.socket.Socket.r

listeninglistening

